***Background:***

Steve Rummel, Senior Manager, Data Analytics & Digitization, created the below Python training course as part of his personal study of Python and development of training materials for business users of Python, on his own time and using his own resources prior to his employment at CVS. He granted permission to the the CVS Health Internal Audit department to adapt the case study into a more robustly documented course to help employees upskill in Python. Additional information in the commented sections has also been added by Alan Harrington, Data Scientist, for clarification and organizational purposes.

***Scenario:***

Your supervisor asks you to perform the following based on data they have collected as part of an audit:
1) Clean and consistently format the data;\
2) Combine the data together;\
3) Identify all unique customers and customer IDs;\
4) Summarize the data to show how many people are in each client;\
5) Identify records with a service expiration date prior to system date;\
6) Show the top 5 records with customer expense greater than the client threshold;\
7) Summarize the data by client name and amount where customer expenses exceed the client threshold; and\
8) Graph the data by distribution of Count of Customers by Birth Year (You’ll need to use matplotlib).

***Notes:***

Note 1: The Python script below creates fake member data using the Faker library. Any semblance to real life individuals is purely coincidental.

Note 2: Some functions were taken and adapted from various websites, including caktusgroup.com and stackoverflow.com. Any instances where functions were created that were not authored by Steve Rummel have been cited appropriately.

Note 3: If a user is an Aetna user, reference the Aetna Nexus respository to ensure that the Python Libraries used herein are still able to be used.  

Note 4: Any recommendations/feedback should be sent to Steve Rummel, steven.rummel@cvshealth.com, ***and*** Alan Harrington, harringtona@aetna.com. 

Note 5: Faker specific documentation can be found at https://faker.readthedocs.io/en/master/ for additional options to customize the script below

Note 6: Pandas specific documentation can be found at https://pandas.pydata.org/.

Note 7: NumPy specific documentation can be found at https://numpy.org/.

Note 8: DateTime specific documentation can be found at https://docs.python.org/3/library/datetime.html.

Note 9: Pathlib specific documentation can be found at https://docs.python.org/3/library/pathlib.html.

Note 10: Matplotlib specific documentation can be found at https://matplotlib.org/stable/tutorials/introductory/pyplot.html.

Note 11: String specific documentation can be found at https://docs.python.org/3/library/string.html

Note 12: CSV specific documentation can be found at https://docs.python.org/3/library/csv.html

Note 13: Random specific documentation can be found at https://docs.python.org/3/library/random.html

Note 14: Users should familiarize themselves with the library documentation above.

Note 15: In the 'Output' folder, there should be 9 files output:
1. CLIENTS_MASTER.csv
2. Data_MASTER.csv
3. Data_MASTER.pkl
4. File_1.csv
5. File_2.csv
6. File_3.csv
7. File_4.csv
8. File_5.csv
9. SOLUTION_DATASET.csv

Note 16: Rerunning the script will just overwrite the outputs.

In [1]:
# Import the required libraries and modules.
# Note, some users may not have some of these pre-installed.
# Verify with the nexus to ensure correct version installation of libraries.

import pandas as pd
import numpy as np
import datetime
import string
import random
import csv
from datetime import timedelta as td
from pathlib import Path
from faker import Faker
from faker.providers.person import Provider
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#==========Establishing random seed, directory pathing, start date, end date, and size.==========

# The user will need to use numpy to specify a specific random seed number to generate their results.
# This will allow results to be reproducible and still random no matter the run.

# The below uses NumPy to generate the random seed
np.random.seed(42)

# Create a destination directory 'dest_dir' for the files to be output to.
# Recall, only 'Path' was imported from 'pathlib' as such, we initiate the input of the 'dest_dir' by 
# using Path.cwd() to return a new path object representing the current working directory (cwd for short).
# '.joinpath()' is equivalent to combining the path with each of the other arguments in turn and, as such,
# the output files will be sent only to the folder named 'Output'.
# Note, this destination directory variable will be used in the following lines of code so if this variable name
# is changed, the user MUST update the subsequent code!
dest_dir = Path.cwd().joinpath('Output')

# mkdir creates a new directory at a given path, in this case, the dest_directory path
# parents = True means that any missing parents of this path are created as needed. So if an output folder
# was not present, an Output folder would be created.
# exist_ok means that the error code FileExistsError, will be ignored only if the last path component is not 
# an existing non-directory file
Path(dest_dir).mkdir(parents=True, exist_ok=True)

# Create the 'end_date' variable and store within it the current date using pd.to_datetime('today').
# This returns the current date at midnight, irrespective of when the script is run.
# Note, you have to use .normalize() after to return the time portion as 00:00:00
# E.g., if this is run on July 5, 2023 at 1:46PM, the 'end_date' would appear as 
# Timestamp('2023-07-05 13:46:16.133163'). However, if adding .normalize() to the end
# it instead returns Timestamp('2023-07-05 00:00:00').
# Thus the dataset's end date will be relative to the date the script is run.
end_date = pd.to_datetime('today').normalize()

# Create a start date based on our oldest customer being 100 years old
# relative to the end_date specified above.
start_date = end_date - datetime.timedelta(days=100 * 365)

# Set a size for our resulting dataset in records.
# Note: For training purposes, this should be set to a 100,000-500,000 range.
size = 100000

#==========Create the correct localizations using Faker.==========

# Faker has multiple localizations - places one can get names for. Mexico, the US, etc.
# We will use several locationlizations to obtain an diverse set of names, including those with
# non-standard/unicode characters. The user will need to understand how to deal with these variations.

# Loading the localizations specified into the localizations variable 
localizations = ['it_IT', 'en_US', 'es_ES', 'es_MX', 'fi_FI', 'nl_NL', 'en_IN', 'fr_FR']

# Use 'Faker()' to initialize the generation of names with the above localizations
fake = Faker(localizations)

# To obtain only US addresses for our members, we create a new Faker generation instance and load it into a variable.
# If you use 'fake' instead of 'addr' as a variable, all the names will appear as what the Faker library
# creators consider "normal" American names, which mostly rely on popular American names from the mid-20th century.
addr = Faker('en_US')

#==========Create a tuple containing client_names.==========

# We will create 6 client names, one of which is a TEST_CLIENT. The 'TEST_CLIENT' has been purposefully input
# into the list of clients below to prepare users for looking for abnormal values in their data.
# Note, the client_names variable is a tuple.

client_names = ("Acme Health Care",
                "Massive Dynamic Employee HC",
                "Federal Hornswogglers Union HC",
                "Statler & Waldorf Investment Group",
                "Amazing Distribution Co. Employee HC",
                "TEST_CLIENT"
                )

# The below will create a few functions. Functions are created by
# using 'def' followed by the name of the function then (), with any parameters noted within the ().


def random_dates(year, size):
    '''The purpose of this function is to create random dates within a range between the start and end dates.
    Adapted from: https://stackoverflow.com/a/50668285.'''
    #Note, Unix timestamp is in nanoseconds by default, so divide it by
    # 24*60*60*10**9 to convert to days.
    # format=specifies the format in which the date will be parsed out as
    # errors= inin this case, by default it raises an exception
    start_u=pd.to_datetime('{}0101'.format(year), format="%Y%m%d", errors='raise')
    end_u=pd.to_datetime('{}1231'.format(year), format="%Y%m%d", errors='raise')
    divide_by = (24 * 60 * 60 * 10**9)
    # Convert the start and end times accordingly
    start_u = start_u.value // divide_by
    end_u = end_u.value // divide_by
    # Return a datetime that has a random integer based on the predefined start_u, end_u, and 
    # size paramters with a unit "D" for days.
    return pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")


def get_random_birthdates(start, end, size=10, mean=50, sd=10):
    '''The purpose of this function is to create birthdates based on our above coded relative start and end dates.
    The function assumes a normally distributed range of ages from 0 to 100 years old with a mean
    of 50 and a standard deviation (sd) of 10, which gets us close to a noraml distrubtion of 100 years'''
    num_people=10000
    end_date = end #end = pd.to_datetime('today').normalize()
    start_date = start #start = (end_date - datetime.timedelta(days=time_interval * 365))
    # Returns the interval in both days and years 
    interval_in_days=(end_date - start_date).days   
    interval_in_years=(end_date.year - start_date.year)   

    # Create the variable 'df' and load into it a dataframe using the using integer variables 
    # rounded as whole numbers to pull random samples from a normal (Gaussian) distribution.
    df = pd.DataFrame(np.random.normal(mean, sd, size).round(0).astype(int))
    # Create a count of the dataframe as a grouped summary 
    summary = df[0].groupby(df[0]).count()

    # Create an empty list named 'new_birthday_list'
    new_birthday_list=[]
    # Create a loop that for every year within the range (noted as 0 years to the 
    # interval_in_years as calculated above ) 
    for year in range(0, interval_in_years):
        # Set the current year in the loop as the start_date year plus the year so 
        # it'll loop through every year.
        current_year=start_date.year + year
        # Try and except within a loop will try the below code first and if 
        # it does not work then the except code will be executed. 
        try:
            # load the count of records for the year into a variable named 'count'
            count=summary[year]
            # use the random_dates function above and input the current_year and count as parameter values
            # and store all values into a variable named 'test_dates'
            test_dates=random_dates(current_year, count)
            # Create another loop where for each date within the tests_dates
            for date in test_dates:
                # append each date to the new_birthday_list variable above
                new_birthday_list.append(date)
        except KeyError:
            # in the instance where the try part of the loop cannot be executed, in the instance
            # of a KeyError, put the count to 0
            count=0
    #Return the new_birthday_list as the output of the function
    return new_birthday_list


def generate_random_values(size, probabilities, categories):
    """Generate size-length ndarray of categories based on probabilities."""
    return np.random.choice(categories, size=size, p=probabilities)

def assign_gender_based_name(gender):
    '''The purpose of this function checks the gender value and returns a specific name value in accordance
    with the Faker library results.'''
    # Note, this means that it will still generate names for O and Empty values in the data
    if gender=='M':
        retval=fake.name_male()
    elif gender=='F':
        retval=fake.name_female()
    else:
        retval=fake.name()
    return retval

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    '''The purpose of this function is to create a string ID for each member record.
    Source of the function:
    From: https://stackoverflow.com/questions/2257441/random-string-generation-with-upper-case-letters-and-digits/2257449'''
    return ''.join(random.choice(chars) for _ in range(size))


def descending_probabilities(value):
    """The purpose of this function is to generate a set of probabilities.
    If we do not have a specific set of probabilities in mind for whatever
    fake set of data elements we are populating, we can simply use this to
    get a set of probabilities that will be allocated to the elements in
    descending order of the proportion of the total number of elements
    in a 'sum of the digits' manner."""
    inc = 0
    values =[]
    agg_prob=0
    for i in range(value, 0, -1):
        inc += i
    for j in range(value, 1, -1):
        prob=round(j/inc, 2)
        values.append(prob)
        agg_prob=agg_prob + prob
    values.append(round(1-agg_prob, 2))
    return values


print("Done. Start date: {} End date: {}".format(start_date, end_date))

Done. Start date: 1923-08-01 00:00:00 End date: 2023-07-07 00:00:00


In [3]:
#==========Create a plans table with maximum deductible and plan ID.==========

# Create a blank dataframe and load the dataframe into the 'clients' variable
clients=pd.DataFrame()
# Create a column named 'CLIENT_NAME' and populate with the 6 client_names
clients['CLIENT_NAME']=client_names

# Create a column named 'CLIENT_ID' and populate with the value 6. 
# This will populate all values in each row with 6
clients['CLIENT_ID'] = 6

# Apply the 'id_generator' function on the CLIENT_ID field to overwrite the 6 values and
# replace with a new ID
clients['CLIENT_ID'] = clients['CLIENT_ID'].apply(id_generator)

# Create a variable named 'client_bal_dist' and populate with a random 
# number based on a normal (Gaussian) distribution with a scale/standard deviation (spread or "width")
# of 100 and a mean ("centre") of distribution/loc of 1,500, and a size equal to the index of the clients 
# variable and round the number to two.
# This SHOULD return a NumPy array of 6 numbers  
client_bal_dist=np.random.normal(size = len(clients.index), scale = 100, loc = 1500).round(2)

# Use the values from the 'client_bal_dist' variable as values in a new column named 'THRESHOLD'
# for the 'clients' dataframe. 
clients['THRESHOLD']=client_bal_dist

# Export the clients data to a file named 'CLIENTS_MASTER.csv'
clients.to_csv(dest_dir.joinpath('CLIENTS_MASTER.csv'), index=False)

#==========Create customer data using our plan data.==========

# Next, create our customers data using our plan data.

# Create a blank dataframe.
df = pd.DataFrame()

# Assign a random plan identifier.

# Load only the unique values from the CLIENT_ID column of clients into a variable named 'client_ids' 
client_ids=clients['CLIENT_ID'].unique()

# Use the descending_probabilities function to generate a list of probabilities that are the same length 
# as the client_ids variable
plan_probabilities=descending_probabilities(len(client_ids))

# Create a column named 'Client_ID' in the 'df' dataframe and generate records
# using the size as previously specified above, 100,000, set the probabilities
# to plan_proabilities as defined above, and set the categories to client_ids.
df['Client_ID'] = generate_random_values(size, probabilities=plan_probabilities, categories=client_ids)

# Set a specific probability for each of the plans to appear wtihin the data
p = (0.30, 0.30, 0.10, 0.05, 0.24, 0.01)
plans = ("A", "B", "C", "D", "F", "TEST")
# Use the generate_random_values function to generate values for a new field within
# the 'df' dataframe named 'Plan_ID'.
df['Plan_ID'] = generate_random_values(size, probabilities=p, categories=plans)

# Create a field named 'Customer_ID' in the 'df' dataframe and populate with the value 9. 
# There is likely a better way to do this, but we need a random and
# unique 'user id' so create one that is 9 alpha-numerals long.
df['Customer_ID'] = 9

# Apply the 'id_generator' function on the Customer_ID field to overwrite the 9 values and
# replace with a new ID
df['Customer_ID'] = df['Customer_ID'].apply(id_generator)

# Create a field named 'ACCOUNT_STATUS' for the 'df' dataframe and 
# populate the field using the generate_random_values function
# with the size equal to 100,000, as described above, probabilities set to 
# 94% and 6%, with categories of 'ACTIVE' and 'INACTIVE', respectively.
df['ACCOUNT_STATUS'] = generate_random_values(size, probabilities=(.94, .06), categories=('ACTIVE', 'INACTIVE'))

# Create a field named 'Gender' for the 'df' dataframe and 
# populate the field using the generate_random_values function
# with the size equal to 100,000, as described above, probabilities set to 
# 49%, 49%, 1%, and 1%, with categories of "M", "F", "O", and "", respectively.
# The "" represents a blank.
# Trying to be inclusive, so added Other and blanks. 
df['Gender'] = generate_random_values(size, probabilities=(0.49, 0.49, 0.01, 0.01), categories=("M", "F", "O", ""))

# Create a field named 'Full_Name' for the 'df' dataframe to create
# names strictly based on the Faker modules population of names. 
df['Full_Name'] = df['Gender'].apply(assign_gender_based_name)

# Create a variable called 'addresses' and populate with an empty list, [].
addresses=[]
# Note, we have limited the addresses to only English characters for names, 
# but this can be changed if desired. 
for _ in range(size):
    addresses.append(addr.address())

# Also, using list comprehension we have replaced instances 
# where the address would move to a new line with ', ' and stored 
# the addresses into a new field within the 'df' dataframe.
df['Address'] = [w.replace('\n', ', ') for w in addresses]

# Create the 'Birthdate' column in the 'df' dataframe using the get_random_birthdates
# function using the start, end, and size parameters specified previously and with
# a mean of 50 and standard deviation of 10.
df['Birthdate'] = get_random_birthdates(start=start_date, end=end_date, size=size, mean=50, sd=10)
# Create several birthdate fields of different formats to force students convert
# each birthdate field to the same date format.
df['Birthdate_01'] = df['Birthdate'].dt.strftime('%m/%d/%Y')
df['Birthdate_02'] = df['Birthdate'].dt.strftime('%m%d%y')
df['Birthdate_03'] = df['Birthdate'].dt.strftime('%m-%d-%Y')
df['Birthdate_04'] = df['Birthdate'].dt.strftime('%Y%m%d')

# Assign to the variable 'cust_bal_dist' variable random samples from a normal 
# (Gaussian) distribution, with a size equal to the index of the 'df' dataframe,
# a scale/standard deviation (spread or "width") of 200, and a mean ("center") 
# of distribution/loc of 1,200, rounded to two decimal places.
cust_bal_dist=np.random.normal(size = len(df.index), scale = 200, loc = 1200).round(2)

# Create the field 'CUST_EXP' for the 'df' dataframe and populate it with values from
# the cust_bal_dist variable.
df['CUST_EXP']=cust_bal_dist

# Create a field named 'SYSTEM_DATE' in the 'df' dataframe and populate it with
# values datetime values using the end_date variable.
# This will provide a Baseline date against which we generate everything else, 
# so we have relative ranges, errors, etc.
df['SYSTEM_DATE']=pd.to_datetime(end_date)

# Assign to the variable 'dist' variable random samples from a normal 
# (Gaussian) distribution, with a size equal to the index of the 'df' dataframe,
# a scale/standard deviation (spread or "width") of 100, and the arithmetic 
# mean ("center") of distribution/loc of 190, rounding the values and ensuring the
# values are an integer.

dist=np.random.normal(size = size, scale = 100, loc = 190)
dist=dist.round().astype(int)

# Create a field named 'Service_Expiration_Date' in the 'df' dataframe.
# We will add values to the original 'SYSTEM_DATE' field to ensure that,
# for educational purposes, ~3-5% of our population are failures, which
# represent instances where the Service_Expiration_Date occurred before
# the SYSTEM_DATE. The Service_Expiration_Date field is populated by taking
# the original SYSTEM_DATE value and adding to it the timedelta based
# on the 'dist' variable calculated above in days.
df['Service_Expiration_Date'] = df['SYSTEM_DATE'] + pd.to_timedelta(dist, unit='days')

#export the 'df' data to both a pickle file and a csv file
df.to_pickle(dest_dir.joinpath('Data_MASTER.pkl'))
df.to_csv(dest_dir.joinpath('Data_MASTER.csv'))

#print the minimum and maximium birthdates
print("Done! minimum birthdate: {} maximum birthdate: {}".format(min(df['Birthdate']), max(df['Birthdate'])))


Done! minimum birthdate: 1930-07-16 00:00:00 maximum birthdate: 2016-11-24 00:00:00


In [4]:
# Export into a set of files, using different birthday fields for each.

# Create a list of the Birthdate columns previously created
bdates = ['Birthdate', 'Birthdate_01', 'Birthdate_02', 'Birthdate_03', 'Birthdate_04']

# Obtain the length of the list of bdates list into the 'iter_inc' variable
iter_inc=len(bdates)

# Obtain the length of the 'df' dataframe and load into the 'df_len' variable
df_len = len(df.index)

# Create an empty listing that and load into the 'result' variable
result = []

# Create a list of fields to retain, which will drop all non-necessary fields
master_retain_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP', 
                        'Gender', 'Full_Name', 'Address',
                        'Birthdate', 'Service_Expiration_Date', 'SYSTEM_DATE']

# Create the variable 'numfiles' (number of files) and set the value to 5
numfiles=5

# Create the variable 'inc' and set the value to 0
inc=0

# Create the variable 'start' and set the value to 0
start=0

# For file rows in range(), range has 3 parameters, start, stop, and step.
# In this instance, the start is at 0, the stop is df_len+1 (this is due
# to the stop parameter stopping 1 before the number specified), and
# the step (which is the rounded number of the length of the dataframe
# divided by the number of files, thus splitting the records per file 
# evenly between each saved file).
for filerows in range(0, df_len+1, round(df_len/numfiles)):
    # The modulus below is allowing us to select a different birthday
    # field for each output file and cycle through, so if there are
    # ten output files, each of the five fields will be used in 2 
    # or three of them forcing users to format them correctly.
    bday_field=bdates[inc % iter_inc]
    
    # Retains only the fields specified. Note, this is different than the 
    # master_retain_fields due to needing to substitute a field named 
    # Birthdate for the bday_field variable that is dynamic enough to 
    # cycle through the birthdate field name list
    retain_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP',
                     'Gender', 'Full_Name', 'Address',
                     bday_field, 'Service_Expiration_Date', 'SYSTEM_DATE']
    
    # If the file rows are greater than start, start being 0 as noted above
    # upon the first run. 
    if filerows > start:
        
        # load into a variable named 'fname' the name of the csv file, being 
        # dynamic as a f-string to allow it to input the number of the 
        # file, e.g., File_1.csv, File_2.csv, etc.
        fname='File_{}.csv'.format(inc)
        
        #print the name of each file as it is being saved. This message
        # will appear at the bottom of the cell.
        print("Saving {}...".format(fname))
        
        # For retained fields in the 'df' dataframe, use integer-location (iloc)
        # to slice the array from the start, 0 in the case of file_1.csv, to
        # the end of filerows, and export as a csv which will be output
        # to the 'dest_dir' path, using the retain_fields 'header' variable
        # keeping the index as False (which does not write row names), and
        # set quoting to csv.QUOTE_ALL to quote everything regardless of field type.
        df[retain_fields].iloc[start:filerows].to_csv(dest_dir.joinpath(fname),
                                                      header=retain_fields,
                                                      index=False,
                                                      quoting=csv.QUOTE_ALL)
    # Replace the original value of the start variable with the filerows integer
    start=filerows
    # Replace the value of 'inc' by adding 1 to the 'inc' variable every iteration
    inc += 1
   
# Print a message to note the number of records that should have been exported.
# This serves as a double check to the user to compare the code to the output.
print("Should have exported {} records.".format(df_len))

# Print "Done!" as a final message to know when the code has finished running.
print("Done!")

Saving File_1.csv...
Saving File_2.csv...
Saving File_3.csv...
Saving File_4.csv...
Saving File_5.csv...
Should have exported 100000 records.
Done!


In [5]:
# Create a variable 'header_fields' variable with the correct final fields
# to be used for the solution set.
header_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP',
                 'Gender', 'Full_Name', 'Address',
                 'CUSTOMER_BIRTHDAY', 'Service_Expiration_Date', 'SYSTEM_DATE']

# Print the message, "Exporting solution dataset..."
print("Exporting solution dataset...")

# Format the 'df' dataframe fields to only the 'header_fields' above and
# export to a csv file named 'SOLUTION_DATASET.csv', not writing 
# row names (index=false) and quoting everything.
df[master_retain_fields].to_csv(dest_dir.joinpath('SOLUTION_DATASET.csv'),
                                header=header_fields,
                                index=False,
                                quoting=csv.QUOTE_ALL)
print("Done!")

Exporting solution dataset...
Done!


***The below is the above code all formatted in one cell without the comments to serve as an example of the whole code being processed at once. Users can run this cell as needed. Note, whenever writing code some comments should be left to help with review of documentation, but not to the same degree as the above.***

In [6]:
import pandas as pd
import numpy as np
import datetime
import string
import random
import csv
from datetime import timedelta as td
from pathlib import Path
from faker import Faker
from faker.providers.person import Provider
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
dest_dir = Path.cwd().joinpath('Output')
Path(dest_dir).mkdir(parents=True, exist_ok=True)
end_date = pd.to_datetime('today').normalize()
start_date = end_date - datetime.timedelta(days=100 * 365)
size = 100000
localizations = ['it_IT', 'en_US', 'es_ES', 'es_MX', 'fi_FI', 'nl_NL', 'en_IN', 'fr_FR']
fake = Faker(localizations)
addr = Faker('en_US')
client_names = ("Acme Health Care",
                "Massive Dynamic Employee HC",
                "Federal Hornswogglers Union HC",
                "Statler & Waldorf Investment Group",
                "Amazing Distribution Co. Employee HC",
                "TEST_CLIENT"
                )

def random_dates(year, size):
    start_u=pd.to_datetime('{}0101'.format(year), format="%Y%m%d", errors='raise')
    end_u=pd.to_datetime('{}1231'.format(year), format="%Y%m%d", errors='raise')
    divide_by = (24 * 60 * 60 * 10**9)
    start_u = start_u.value // divide_by
    end_u = end_u.value // divide_by
    return pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")

def get_random_birthdates(start, end, size=10, mean=50, sd=10):
    num_people=10000
    end_date = end 
    start_date = start 
    interval_in_days=(end_date - start_date).days   
    interval_in_years=(end_date.year - start_date.year)   
    df = pd.DataFrame(np.random.normal(mean, sd, size).round(0).astype(int))
    summary = df[0].groupby(df[0]).count()
    new_birthday_list=[]
    for year in range(0, interval_in_years):
        current_year=start_date.year + year
        try:
            count=summary[year]
            test_dates=random_dates(current_year, count)
            for date in test_dates:
                new_birthday_list.append(date)
        except KeyError:
            count=0
    return new_birthday_list

def generate_random_values(size, probabilities, categories):
    return np.random.choice(categories, size=size, p=probabilities)

def assign_gender_based_name(gender):
    if gender=='M':
        retval=fake.name_male()
    elif gender=='F':
        retval=fake.name_female()
    else:
        retval=fake.name()
    return retval

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))


def descending_probabilities(value):
    inc = 0
    values =[]
    agg_prob=0
    for i in range(value, 0, -1):
        inc += i
    for j in range(value, 1, -1):
        prob=round(j/inc, 2)
        values.append(prob)
        agg_prob=agg_prob + prob
    values.append(round(1-agg_prob, 2))
    return values

print("Done. Start date: {} End date: {}".format(start_date, end_date))

clients=pd.DataFrame()
clients['CLIENT_NAME']=client_names
clients['CLIENT_ID'] = 6
clients['CLIENT_ID'] = clients['CLIENT_ID'].apply(id_generator)
client_bal_dist=np.random.normal(size = len(clients.index), scale = 100, loc = 1500).round(2)
clients['THRESHOLD']=client_bal_dist
clients.to_csv(dest_dir.joinpath('CLIENTS_MASTER.csv'), index=False)
df = pd.DataFrame()
client_ids=clients['CLIENT_ID'].unique()
plan_probabilities=descending_probabilities(len(client_ids))
df['Client_ID'] = generate_random_values(size, probabilities=plan_probabilities, categories=client_ids)
p = (0.30, 0.30, 0.10, 0.05, 0.24, 0.01)
plans = ("A", "B", "C", "D", "F", "TEST")
df['Plan_ID'] = generate_random_values(size, probabilities=p, categories=plans)
df['Customer_ID'] = 9
df['Customer_ID'] = df['Customer_ID'].apply(id_generator)
df['ACCOUNT_STATUS'] = generate_random_values(size, probabilities=(.94, .06), categories=('ACTIVE', 'INACTIVE'))
df['Gender'] = generate_random_values(size, probabilities=(0.49, 0.49, 0.01, 0.01), categories=("M", "F", "O", ""))
df['Full_Name'] = df['Gender'].apply(assign_gender_based_name)
addresses=[]

for _ in range(size):
    addresses.append(addr.address())

df['Address'] = [w.replace('\n', ', ') for w in addresses]
df['Birthdate'] = get_random_birthdates(start=start_date, end=end_date, size=size, mean=50, sd=10)
df['Birthdate_01'] = df['Birthdate'].dt.strftime('%m/%d/%Y')
df['Birthdate_02'] = df['Birthdate'].dt.strftime('%m%d%y')
df['Birthdate_03'] = df['Birthdate'].dt.strftime('%m-%d-%Y')
df['Birthdate_04'] = df['Birthdate'].dt.strftime('%Y%m%d')
cust_bal_dist=np.random.normal(size = len(df.index), scale = 200, loc = 1200).round(2)
df['CUST_EXP']=cust_bal_dist
df['SYSTEM_DATE']=pd.to_datetime(end_date)
dist=np.random.normal(size = size, scale = 100, loc = 190)
dist=dist.round().astype(int)
df['Service_Expiration_Date'] = df['SYSTEM_DATE'] + pd.to_timedelta(dist, unit='days')
df.to_pickle(dest_dir.joinpath('Data_MASTER.pkl'))
df.to_csv(dest_dir.joinpath('Data_MASTER.csv'))
print("Done! minimum birthdate: {} maximum birthdate: {}".format(min(df['Birthdate']), max(df['Birthdate'])))
bdates = ['Birthdate', 'Birthdate_01', 'Birthdate_02', 'Birthdate_03', 'Birthdate_04']
iter_inc=len(bdates)
df_len = len(df.index)
result = []
master_retain_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP', 
                        'Gender', 'Full_Name', 'Address',
                        'Birthdate', 'Service_Expiration_Date', 'SYSTEM_DATE']
numfiles=5
inc=0
start=0

for filerows in range(0, df_len+1, round(df_len/numfiles)):
    bday_field=bdates[inc % iter_inc]
    retain_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP',
                     'Gender', 'Full_Name', 'Address',
                     bday_field, 'Service_Expiration_Date', 'SYSTEM_DATE']
    if filerows > start:
        fname='File_{}.csv'.format(inc)
        print("Saving {}...".format(fname))
        df[retain_fields].iloc[start:filerows].to_csv(dest_dir.joinpath(fname),
                                                      header=retain_fields,
                                                      index=False,
                                                      quoting=csv.QUOTE_ALL)
    start=filerows
    inc += 1

print("Should have exported {} records.".format(df_len))

header_fields = ['Customer_ID', 'Client_ID', 'Plan_ID', 'CUST_EXP',
                 'Gender', 'Full_Name', 'Address',
                 'CUSTOMER_BIRTHDAY', 'Service_Expiration_Date', 'SYSTEM_DATE']
print("Exporting solution dataset...")
df[master_retain_fields].to_csv(dest_dir.joinpath('SOLUTION_DATASET.csv'),
                                header=header_fields,
                                index=False,
                                quoting=csv.QUOTE_ALL)
print("Done!")

Done. Start date: 1923-08-01 00:00:00 End date: 2023-07-07 00:00:00
Done! minimum birthdate: 1930-07-16 00:00:00 maximum birthdate: 2016-11-24 00:00:00
Saving File_1.csv...
Saving File_2.csv...
Saving File_3.csv...
Saving File_4.csv...
Saving File_5.csv...
Should have exported 100000 records.
Exporting solution dataset...
Done!
